# Approximated Closeness Centrality

### Libraries import & graph retrieval

In [3]:
import pickle               # to read the Amazon DiGraph
import networkx as nx       # to work with graphs

graph_path = "../data/processed/amazon_graph.pickle"

with open(graph_path, "rb") as f:
    G = pickle.load(f)

---

## Hyperball Algorithm

HyperBall è l'algoritmo che usa i contatori HLL per calcolare la centralità su tutto il grafo in modo efficiente. L'idea geniale è che non serve fare una visita (BFS) per ogni nodo. Si possono "muovere" i contatori.

Il Funzionamento Passo-Passo:
1. Inizializzazione ($t=0$): Ogni nodo ha un contatore HLL che contiene solo se stesso. Quindi ogni nodo sa di poter raggiungere 1 persona (se stesso) a distanza 0.
2. Iterazione ($t=1, 2, \dots$): Immagina che ogni nodo dica ai suoi vicini: "Ehi, questi sono tutti i nodi che io so raggiungere". In termini tecnici: il contatore HLL di un nodo $x$ viene aggiornato facendone l'unione con i contatori HLL dei suoi vicini. Grazie alla matematica degli HLL, l'unione è un'operazione velocissima (massimizzazione dei registri).
3. Stima delle Distanze: Al passo $t$, il contatore del nodo $x$ stima quanti nodi sono raggiungibili in $\le t$ passi (chiamiamo questa stima $N_t$). Al passo precedente ($t-1$), sapevi quanti erano raggiungibili in $\le t-1$ passi ($N_{t-1}$). La differenza $N_t - N_{t-1}$ ti dice approssimativamente quanti nodi sono esattamente a distanza $t$.
4. Calcolo della Centralità: Sapendo quanti nodi sono a distanza $t$, puoi aggiungere quel numero moltiplicato per $\frac{1}{t}$ al totale della tua Harmonic Centrality.

### Dettaglio del funzionamento di HyperBall
Quando hai pochi nodi (1, 10, 100), l'algoritmo funziona in modo diverso rispetto a quando ne hai milioni. Non guarda tanto l'altezza degli zeri, ma **quanti registri sono rimasti a zero**.

Immagina di avere **1024 secchi** (i registri) allineati in un campo sotto la pioggia.
Ogni nodo è una goccia di pioggia che cade in un secchio casuale (determinato dai primi 10 bit dell'hash).

* **1 Nodo (1 goccia):** Bagna 1 secchio. 1023 secchi sono asciutti (valore 0).
* **2 Nodi (2 gocce):** Bagnano probabilmente 2 secchi diversi. 1022 secchi asciutti.
* **100 Nodi (100 gocce):** Bagnano circa 90-95 secchi. ~930 secchi sono ancora asciutti.

**La matematica della Media Armonica ():**
La formula che usi (`sum(2**-M)`) è sensibilissima ai valori bassi.

* Se un registro è vuoto (valore 0), contribuisce alla somma con .
* Se un registro ha un valore alto (es. 10), contribuisce pochissimo ().

Se hai 1 nodo "fortunato" che finisce nel registro #5 con valore 10, ma gli altri 1023 registri sono vuoti (valore 0), la somma sarà:



*(Nota: senza una correzione specifica chiamata "Linear Counting", l'HLL puro sovrastima un po' i numeri piccolissimi, ma l'ordine di grandezza rimane contenuto grazie alla massa di registri a zero).*

Quindi: **Finché ci sono tanti registri a zero, la stima rimane bassa**, indipendentemente da quanto è "fortunato" quel singolo nodo.

### 2. La transizione ai Grandi Numeri

Man mano che i nodi aumentano, i secchi si riempiono tutti. Non ci sono più registri a zero.
A questo punto, l'algoritmo smette di basarsi sui "vuoti" e inizia a basarsi sulla **rarità**.

* Con 10.000 nodi, è probabile che in ogni secchio sia caduta almeno una goccia.
* Ora guardiamo **dentro** ogni secchio: qual è la goccia "più rara" (più leading zeros) che è caduta qui?

È qui che entra in gioco la tua intuizione: su 1024 registri, la media dei valori massimi salirà statisticamente in modo molto preciso all'aumentare dei nodi unici.

### 3. Perché la Media Armonica?

L'uso della media armonica (invece di quella aritmetica) serve proprio a **proteggersi dalla fortuna**.

* **Media Aritmetica:** Se su 1024 studenti, 1023 hanno 0 euro e uno ha 1 milione di euro, la media aritmetica dice che sono tutti ricchi (~1000€ a testa). Sbagliato.
* **Media Armonica:** È pesantemente influenzata dai valori bassi. Nello stesso esempio, la media armonica direbbe che la ricchezza è vicina a 0.

Nell'HyperBall:
Se un nodo ha un hash rarissimo (30 zeri) ma è l'unico nodo che ho incontrato, gli altri 1023 registri a zero "tirano giù" la stima con una forza enorme, impedendo che quel singolo evento fortunato faccia schizzare la cardinalità stimata a miliardi.

### 4. Precisione e Limiti

Non è *perfetto*. È una stima probabilistica.
Con  registri (), l'errore standard è circa:

Significa che se il numero reale è 100, l'algoritmo potrebbe dirti 97 o 103. Se è 1.000.000, potrebbe dirti 1.030.000.
Per calcolare la centralità in un grafo, questo errore è assolutamente accettabile perché ti interessa l'ordine di grandezza e la classifica relativa dei nodi, non il numero atomico esatto.


### Versione 1: calcolo su CPU usando dizionari e HyperLogLog

In [1]:
from datasketch import HyperLogLog
import copy
import time

def harmonic_v1_CPU(G, p=10):
    """
    Calcola la Harmonic Centrality approssimata usando HyperBall.

    Args:
        G: Oggetto NetworkX DiGraph (350k nodi).
        p: Precisione dei registri (p=10 -> 2^10 registri = errore ~3%).
           Valori più bassi usano meno RAM ma sono meno precisi.
    """

    # =========================================================================
    # FASE 1: PREPARAZIONE E INIZIALIZZAZIONE
    # =========================================================================
    # Per calcolare la centralità "in entrata" (quanto sono importante),
    # dobbiamo contare chi può raggiungere ME. HyperBall propaga "in avanti",
    # quindi lavoriamo sul grafo trasposto (archi invertiti).
    print(f"--- FASE 1: Inversione grafo e Inizializzazione HLL (p={p}) ---")
    G_rev = G.reverse()
    nodes = list(G_rev.nodes())

    # =========================================================================
    # FASE 2: COSTRUZIONE DELLA MATRICE DI CONTATORI
    # =========================================================================

    # Dizionario per i contatori attuali: {nodo: HyperLogLog}
    # Inizialmente ogni nodo conosce solo se stesso (distanza 0).
    counters = {}
    for node in nodes:
        hll = HyperLogLog(p=p)
        # HLL richiede input in bytes. Convertiamo l'ID del nodo.
        node_id_encoded = str(node).encode('utf-8')
        # Aggiungo il nodo stesso al contatore (inizialmente l'insieme dei nodi raggiungibili contiene solo se stesso
        hll.update(node_id_encoded)
        # Aggiungo il contatore al dizionario che associa ogni nodo ad un contatore HyperLogLog
        counters[node] = hll

    # Dizionari per memorizzare i risultati
    # per ogni nodo in nodes, aggiungi al dizionario la coppia chiave-valore (node:0.0)
    harmonic_centrality = {node: 0.0 for node in nodes}

    # Memorizziamo la cardinalità al passo precedente (N_{t-1}).
    # Al tempo t=0, ogni nodo raggiunge solo se stesso, quindi count = 1.
    prev_cardinality = {node: 1.0 for node in nodes}

    print("Inizializzazione completata.")

    # =========================================================================
    # FASE 2: LOOP PRINCIPALE (Espansione della 'Palla') [cite: 771, 778]
    # =========================================================================
    # Iteriamo per t = 1, 2, ... fino a che le stime non cambiano più (stabilizzazione).
    # t rappresenta la distanza (raggiungibili in t passi).
    t = 0
    changed = True

    while changed:
        t += 1
        changed = False
        print(f"--- Inizio Iterazione t={t} ---")
        start_time = time.time()

        # Buffer per i nuovi contatori del passo t
        next_counters = {}

        # =====================================================================
        # FASE 3: UNIONE DEI CONTATORI (Propagazione) [cite: 775, 780]
        # =====================================================================
        # Logica: I nodi che posso raggiungere in t passi sono l'unione di:
        # 1. Quelli che raggiungevo già (me stesso e i vecchi percorsi)
        # 2. Quelli che raggiungono i miei vicini al passo t-1.

        for node in nodes:
            # Copiamo il contatore attuale del nodo (stato t-1)
            # NOTA: copy è necessario perché HLL è mutabile
            hll_new = copy.copy(counters[node])

            # Uniamo con i contatori dei vicini (successori nel grafo trasposto, G_rev.neighbors() = G.successors())
            # Questo simula il passaggio di informazioni "indietro" nel grafo originale
            neighbors = list(G_rev.neighbors(node))

            if neighbors:
                for neighbor in neighbors:
                    # L'operazione di unione HLL è molto veloce (bit-wise OR dei registri)
                    hll_new.merge(counters[neighbor])

            # Salviamo il nuovo stato
            next_counters[node] = hll_new

        # =====================================================================
        # FASE 4: AGGIORNAMENTO METRICA (Calcolo Harmonic)
        # =====================================================================
        # Formula: H(x) = sum [ (N_t - N_{t-1}) / t ]
        # Dove (N_t - N_{t-1}) è la stima dei nodi trovati ESATTAMENTE a distanza t.

        current_change_count = 0

        for node in nodes:
            old_count = prev_cardinality[node]
            new_count = next_counters[node].count()

            # Se la stima è aumentata, abbiamo trovato nuovi nodi a distanza t
            if new_count > old_count:
                delta = new_count - old_count

                # Aggiungiamo il contributo alla centralità armonica
                harmonic_centrality[node] += delta * (1.0 / t)

                # Aggiorniamo la memoria per il prossimo passo
                prev_cardinality[node] = new_count

                # Segnaliamo che c'è stato un cambiamento nel sistema
                changed = True
                current_change_count += 1

        # =====================================================================
        # FASE 5: CHIUSURA ITERAZIONE E CONTROLLO CONVERGENZA [cite: 833]
        # =====================================================================
        # Scambiamo i buffer: i contatori 'next' diventano quelli attuali per il prossimo t
        counters = next_counters

        elapsed = time.time() - start_time
        print(f"Fine t={t}. Nodi aggiornati: {current_change_count}. Tempo: {elapsed:.2f}s")

        # Sicurezza per evitare loop infiniti in grafi patologici,
        # anche se HLL converge tipicamente entro il diametro effettivo del grafo.
        if t > 1000: # Cutoff arbitrario, aumentabile
            print("Raggiunto limite massimo iterazioni.")
            break

    return harmonic_centrality

---

### Versione 2: calcolo su CPU usando solo matrici NumPy

In [1]:
import numpy as np
import hashlib
import time

def harmonic_v2_CPU(G, p=10):
    """
    Versione CPU di HyperBall basata su NumPy.
    """

    # =========================================================================
    # FASE 1: SETUP
    # =========================================================================
    print(f"--- FASE 1: Setup CPU e Hashing ---")

    G_rev = G.reverse()
    nodes = list(G_rev.nodes())
    n_nodes = len(nodes)

    m = 1 << p
    """
    Numero di registri che compongono ciascun contatore: m = 2^p
    """

    node_to_idx = {node: i for i, node in enumerate(nodes)}
    """
    Mappatura nodo -> indice 0..N-1: dizionario del tipo (k, v) = (node, i), con i risultato dell'enumerazione dell' array di nodi
    """

    edges = np.array([(node_to_idx[u], node_to_idx[v]) for u, v in G_rev.edges()], dtype=np.int32)
    """
    Array di coppie (u_index, v_index), una per ogni edge del tipo (u, v) in G_rev
    """

    M = np.zeros((n_nodes, m), dtype=np.int32)
    """
    Matrice [n_nodes x m] dei contatori;
    Dato che un registro deve contenere il numero di leading zeroes di un hash (64 bits), il massimo valore inseribile in ciascun registro sara' < 64 (dato che i primi b bits dell' hash servono a individuare il registro corretto tra gli m), quindi 1 byte (uint8) e' sufficiente (qui usiamo int32 per sicurezza e compatibilità).
    """

    print("Calcolo hash iniziali su CPU...")

    # Pre-calcolo hash per ogni nodo per inizializzare M
    for i, node in enumerate(nodes):
        # Hash del nodo (crea hash con algoritmo md5 e trasforma il risultato in stringa hex, poi converte in intero a partire da base 16 verso base 10)
        h = int(hashlib.md5(str(node).encode('utf8')).hexdigest(), 16)

        # AND binario tra l' hash h e il numero m-1 = (2^10 - 1) = 1023 = sequenza di zeri seguiti da 10 valori '1'
        # il risultato corrisponde agli ultimi 10 bit di h, che selezionano il registro in cui scrivere il numero di leading zeroes
        j = h & (m - 1)

        # Right shift per rimuovere da h gli ultimi 10 bit estratti in precedenza
        w = h >> p

        # Conteggio del numero di trailing zeroes della porzione di hash rimanente (+1)
        rho = 1
        while (w & 1) == 0 and rho < 32: # while l'ultimo bit di w è uno '0':
            w >>= 1
            rho += 1
        M[i, j] = rho

    # =========================================================================
    # FASE 2: PREPARAZIONE DATI
    # =========================================================================
    print(f"--- FASE 2: Preparazione vettori NumPy ---")

    if len(edges) > 0:
        sources = edges[:, 1]
        """ Array monodimensionale ottenuto prendendo SOLO (tutta) la colonna 1 di edges """
        targets = edges[:, 0]
        """ Array monodimensionale ottenuto prendendo SOLO (tutta) la colonna 0 di edges """
    else:
        sources = np.array([], dtype=np.int32)
        targets = np.array([], dtype=np.int32)

    # Coppia di arrays per memorizzare le cardinalità al passo t-1 e t
    # Inizialmente ogni nodo ha cardinalità 1 (se stesso)
    prev_cardinality = np.ones(n_nodes, dtype=np.float32)
    harmonic_centrality = np.zeros(n_nodes, dtype=np.float32)

    alpha_m = 0.7213 / (1 + 1.079 / m)
    factor = alpha_m * (m ** 2)

    print("Dati pronti. Inizio loop CPU.")

    # =========================================================================
    # FASE 3: LOOP PRINCIPALE SU CPU
    # =========================================================================
    t = 0
    changed = True

    while changed:
        t += 1
        changed = False

        # Time
        start_time = time.time()

        M_prev = M.copy()

        # PROPAGAZIONE IN AVANTI DEI CONTATORI
        # se il numero di archi e' > 0, calcola il massimo tra il registro del nodo sorgente e i registri dei nodi target
        if len(sources) > 0:
            # Prendi i registri dei nodi sorgente
            source_registers = M_prev[sources]
            # Aggiorna i nodi target con il massimo
            # np.maximum.at è l'equivalente NumPy di cp.maximum.at
            np.maximum.at(M, targets, source_registers)

        # CONVERSIONI DELLA MATRICE DEI CONTATORI
        M_float = M.astype(np.float32)
        M_bool = M == 0 # nuova matrice booleana che contiene True se il valore corrispondente in M e' 0

        # PRIMA STIMA - MEDIA ARMONICA
        reg_pow = np.power(2.0, -M_float)
        sum_regs = np.sum(reg_pow, axis=1)

        # Gestione divisione per zero (se sum_regs è 0, anche se improbabile con inizializzazione corretta)
        with np.errstate(divide='ignore'):
            estimate_raw = factor / sum_regs

        # CORREZIONE DELLA STIMA TRAMITE LINEAR COUNTING
        num_zeros = np.sum(M_bool, axis=1) # conta quanti registri sono rimasti a 0 per ciascun nodo
        estimate = estimate_raw.copy()
        threshold = 2.5 * m
        # Maschera booleana che determina quali nodi devono usare la correzione linear counting
        mask_lc = (estimate_raw <= threshold) & (num_zeros > 0)

        if np.any(mask_lc):
            # Formula: m * log(m / V)
            # Calcoliamo solo per i nodi nella maschera
            V = num_zeros[mask_lc].astype(np.float32)
            estimate[mask_lc] = m * np.log(m / V)

        # VERIFICA MODIFICHE RISPETTO ALLA STIMA DI CARDINALITA' PRECEDENTE
        diff = estimate - prev_cardinality
        mask_changed = diff > 0.001

        if np.any(mask_changed):
            changed = True
            harmonic_centrality[mask_changed] += diff[mask_changed] * (1.0 / t)
            prev_cardinality[mask_changed] = estimate[mask_changed]

        # Time
        elapsed = time.time() - start_time

        active_nodes = int(np.sum(mask_changed))
        print(f"Fine t={t}. Tempo CPU: {elapsed:.4f}s. Nodi attivi: {active_nodes}")

        if t > 1000:
            break

    # =========================================================================
    # FASE 4: RECUPERO RISULTATI
    # =========================================================================
    print("Calcolo finito. Restituzione dati...")

    final_centrality = harmonic_centrality
    result = {nodes[i]: final_centrality[i] for i in range(n_nodes)}
    return result


---

### Versione 3: calcolo su GPU usando matrici CuPy

In [2]:
import cupy as cp  # <--- Il sostituto di NumPy per GPU
import numpy as np # Serve ancora per il setup iniziale su CPU
import hashlib
import time

def harmonic_v3_GPU(G, p=10):
    """
    Versione CUDA di HyperBall ottimizzata per CuPy.
    """

    # =========================================================================
    # FASE 1: SETUP SU CPU
    # =========================================================================
    print(f"--- FASE 1: Setup CPU e Hashing ---")

    G_rev = G.reverse()
    nodes = list(G_rev.nodes())
    n_nodes = len(nodes)

    m = 1 << p
    """
    Numero di registri che compongono ciascun contatore: m = 2^p
    """

    node_to_idx = {node: i for i, node in enumerate(nodes)}
    """
    Mappatura nodo -> indice 0..N-1: dizionario del tipo (k, v) = (node, i), con i risultato dell'enumerazione dell' array di nodi
    """

    edges = np.array([(node_to_idx[u], node_to_idx[v]) for u, v in G_rev.edges()], dtype=np.int32)
    """
    Array di coppie (u_index, v_index), una per ogni edge del tipo (u, v) in G_rev
    """

    M_cpu = np.zeros((n_nodes, m), dtype=np.int32)
    """
    Matrice [n_nodes x m] dei contatori;
    Dato che un registro deve contenere il numero di leading zeroes di un hash (64 bits), il massimo valore inseribile in ciascun registro sara' < 64 (dato che i primi b bits dell' hash servono a individuare il registro corretto tra gli m), quindi 1 byte (uint8) e' sufficiente.
    """

    print("Calcolo hash iniziali su CPU...")

    # Pre-calcolo hash per ogni nodo per inizializzare M
    for i, node in enumerate(nodes):
        # Hash del nodo (crea hash con algoritmo md5 e trasforma il risultato in stringa hex, poi converte in intero a partire da base 16 verso base 10)
        h = int(hashlib.md5(str(node).encode('utf8')).hexdigest(), 16)

        # AND binario tra l' hash h e il numero m-1 = (2^10 - 1) = 1023 = sequenza di zeri seguiti da 10 valori '1'
        # il risultato corrisponde agli ultimi 10 bit di h, che selezionano il registro in cui scrivere il numero di leading zeroes
        j = h & (m - 1)

        # Right shift per rimuovere da h gli ultimi 10 bit estratti in precedenza
        w = h >> p

        # Conteggio del numero di trailing zeroes della porzione di hash rimanente (+1)
        rho = 1
        while (w & 1) == 0 and rho < 32: # while l'ultimo bit di w è uno '0':
            w >>= 1
            rho += 1
        M_cpu[i, j] = rho

    # =========================================================================
    # FASE 2: TRASFERIMENTO SU GPU
    # =========================================================================
    print(f"--- FASE 2: Spostamento dati su GPU ---")

    M_gpu = cp.asarray(M_cpu)

    if len(edges) > 0:
        sources_gpu = cp.asarray(edges[:, 1])
        """ Array monodimensionale ottenuto prendendo SOLO (tutta) la colonna 1 di edges """
        targets_gpu = cp.asarray(edges[:, 0])
        """ Array monodimensionale ottenuto prendendo SOLO (tutta) la colonna 0 di edges """
    else:
        sources_gpu = cp.array([], dtype=cp.int32)
        targets_gpu = cp.array([], dtype=cp.int32)

    # Coppia di arrays per memorizzare le cardinalità al passo t-1 e t
    # Inizialmente ogni nodo ha cardinalità 1 (se stesso)
    prev_cardinality_gpu = cp.ones(n_nodes, dtype=cp.float32)
    harmonic_centrality_gpu = cp.zeros(n_nodes, dtype=cp.float32)

    alpha_m = 0.7213 / (1 + 1.079 / m)
    factor = alpha_m * (m ** 2)

    del M_cpu, edges

    print("Dati in VRAM. Inizio loop GPU.")

    # =========================================================================
    # FASE 3: LOOP PRINCIPALE SU GPU
    # =========================================================================
    t = 0
    changed = True

    while changed:
        t += 1
        changed = False

        # Time
        cp.cuda.Stream.null.synchronize()
        start_time = time.time()

        M_gpu_prev = M_gpu.copy()

        # PROPAGAZIONE IN AVANTI DEI CONTATORI
        # se il numero di archi e' > 0, calcola il massimo tra il registro del nodo sorgente e i registri dei nodi target
        if len(sources_gpu) > 0:
            # Prendi i registri dei nodi sorgente
            source_registers = M_gpu_prev[sources_gpu]
            # Aggiorna i nodi target con il massimo
            cp.maximum.at(M_gpu, targets_gpu, source_registers)

        # CONVERSIONI DELLA MATRICE DEI CONTATORI
        M_gpu_float = M_gpu.astype(cp.float32)
        M_gpu_bool = M_gpu == 0 # nuova matrice booleana che contiene True se il valore corrispondente in M_gpu e' 0

        # PRIMA STIMA - MEDIA ARMONICA
        reg_pow = cp.power(2.0, -M_gpu_float)
        sum_regs = cp.sum(reg_pow, axis=1)
        estimate_raw = factor / sum_regs

        # CORREZIONE DELLA STIMA TRAMITE LINEAR COUNTING
        num_zeros = cp.sum(M_gpu_bool, axis=1) # conta quanti registri sono rimasti a 0 per ciascun nodo (dato che True e' considerato 1)
        estimate = estimate_raw.copy()
        threshold = 2.5 * m
        # Maschera booleana che determina quali nodi devono usare la correzione linear counting. un elemento del vettore e' true solo se entrambe le condizioni sono verificate (AND bitwise)
        mask_lc = (estimate_raw <= threshold) & (num_zeros > 0)

        if cp.any(mask_lc):
            # Formula: m * log(m / V)
            # Calcoliamo solo per i nodi nella maschera
            V = num_zeros[mask_lc].astype(cp.float32)
            estimate[mask_lc] = m * cp.log(m / V)

        # VERIFICA MODIFICHE RISPETTO ALLA STIMA DI CARDINALITA' PRECEDENTE
        diff = estimate - prev_cardinality_gpu
        mask_changed = diff > 0.001

        if cp.any(mask_changed):
            changed = True
            harmonic_centrality_gpu[mask_changed] += diff[mask_changed] * (1.0 / t)
            prev_cardinality_gpu[mask_changed] = estimate[mask_changed]

        # Time
        cp.cuda.Stream.null.synchronize()
        elapsed = time.time() - start_time

        active_nodes = int(cp.sum(mask_changed))
        print(f"Fine t={t}. Tempo GPU: {elapsed:.4f}s. Nodi attivi: {active_nodes}")

        if t > 1000:
            break

    # =========================================================================
    # FASE 4: RECUPERO RISULTATI
    # =========================================================================
    print("Calcolo finito. Recupero dati dalla GPU...")

    final_centrality = harmonic_centrality_gpu.get()
    result = {nodes[i]: final_centrality[i] for i in range(n_nodes)}
    return result


---

# Calcolo Harmonic Centrality con metodo HyperBall

In [ ]:
import csv

def save_centrality_to_csv(centrality_dict, filename="harmonic_centrality_results.csv"):
    """
    Salva il dizionario delle centralita' in un file CSV, ordinato per importanza decrescente.

    Args:
        centrality_dict (dict): Dizionario {nodo: score}
        filename (str): Nome del file di output
    """
    print(f"Avvio salvataggio su file: {filename}...")

    # 1. Ordinamento dei dati
    # Trasforma il dizionario in una lista di tuple (nodo, score) ordinata per score decrescente
    # key=lambda x: x[1] indica di ordinare in base al valore (lo score)
    sorted_data = sorted(centrality_dict.items(), key=lambda item: item[1], reverse=True)

    # 2. Scrittura su file
    try:
        # newline='' è importante su Windows per evitare righe vuote extra
        with open(filename, mode='w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)

            # Scrittura dell'intestazione (Header)
            writer.writerow(['ASIN', 'Harmonic_Centrality'])

            # Scrittura di tutte le righe in un colpo solo
            writer.writerows(sorted_data)

        print(f"Salvataggio completato con successo! ({len(sorted_data)} righe scritte)")
        print(f"Il nodo più centrale è '{sorted_data[0][0]}' con score {sorted_data[0][1]:.4f}")

    except IOError as e:
        print(f"Errore durante il salvataggio del file: {e}")

In [5]:
if __name__ == "__main__":
    # centrality_scores = harmonic_v1_CPU(G, p=10)
    # centrality_scores = harmonic_v2_CPU(G, p=10)
    centrality_scores = harmonic_v3_GPU(G, p=10)

    # Stampa top 5
    top_5 = sorted(centrality_scores.items(), key=lambda x: x[1], reverse=True)[:5]
    print("Top 5 nodi per Harmonic Centrality:", top_5)
    save_centrality_to_csv(centrality_scores, "../data/processed/harmonic_scores_v1.csv")

NameError: name 'G' is not defined